In [1]:
# run in the root of the repository
%load_ext autoreload
%autoreload 2
 
%cd ../..

/private/home/pfz/09-videoseal/videoseal-dev


/private/home/pfz/miniconda3/envs/img/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from videoseal.utils.display import save_img
from videoseal.utils import Timer
from videoseal.evals.full import setup_model_from_checkpoint
from videoseal.evals.metrics import bit_accuracy, psnr, ssim
from videoseal.augmentation import Identity, JPEG
from videoseal.modules.jnd import JND, VarianceBasedJND

import os
import omegaconf
from tqdm import tqdm
import gc
from PIL import Image

import torch
import torchvision

to_tensor = torchvision.transforms.ToTensor()
to_pil = torchvision.transforms.ToPILImage()

device = "cpu" 

/private/home/pfz/miniconda3/envs/img/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/private/home/pfz/miniconda3/envs/img/lib/python3.12/site-packages/compressai/models/video/google.py:353: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @amp.autocast(enabled=False)


In [4]:
# Directory containing videos
num_imgs = 10
assets_dir = "assets/imgs"
base_output_dir = "outputs"
os.makedirs(base_output_dir, exist_ok=True)

wam = setup_model_from_checkpoint("/checkpoint/pfz/2025_logs/0306_vseal_ydisc_release_bis/_nbits=256/checkpoint600.pth")
wam.blender.scaling_w = 0.2
wam.eval()
wam.to(device)
    
# wam.compile()

# Iterate over all video files in the directory
file = "/private/home/pfz/_images/tahiti.png"
imgs = Image.open(file, "r").convert("RGB")  # keep only rgb channels
imgs = to_tensor(imgs).unsqueeze(0).float()

# Watermark em
outputs = wam.embed(imgs, is_video=False, lowres_attenuation=True)
torch.cuda.synchronize()
# print(f"embedding watermark  - took {timer.stop():.2f}s")

# compute diff
imgs_w = outputs["imgs_w"]  # b c h w
msgs = outputs["msgs"]  # b k
diff = imgs_w - imgs

# save
imgs_aug = imgs_w
outputs = wam.detect(imgs_aug, is_video=False)
metrics = {
    "file": file,
    "bit_accuracy": bit_accuracy(
        outputs["preds"][:, 1:],
        msgs
    ).nanmean().item(),
    "psnr": psnr(imgs_w, imgs).item(),
    "ssim": ssim(imgs_w, imgs).item()
}

# Augment video
# print(f"compressing and detecting watermarks")
for qf in [80, 40]:
    imgs_aug, _ = JPEG()(imgs_w, None,qf)

    outputs = wam.detect(imgs_aug, is_video=True)
    preds = outputs["preds"]
    # print(preds)
    bit_preds = preds[:, 1:]  # b k ...
    bit_accuracy_ = bit_accuracy(
        bit_preds,
        msgs
    ).nanmean().item()
    
    metrics[f"bit_accuracy_qf{qf}"] = bit_accuracy_

print(metrics)

Model loaded successfully from /checkpoint/pfz/2025_logs/0306_vseal_ydisc_release_bis/_nbits=256/checkpoint600.pth with message: <All keys matched successfully>
{'file': '/private/home/pfz/_images/tahiti.png', 'bit_accuracy': 1.0, 'psnr': 47.33894729614258, 'ssim': 0.9984757304191589, 'bit_accuracy_qf80': 0.99609375, 'bit_accuracy_qf40': 1.0}


In [5]:
# create torchscript for wam.embedder

class EmbedderWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, imgs):
        return self.model.embed(imgs, is_video=False, lowres_attenuation=True)

In [6]:
embedder = EmbedderWrapper(wam)

# save
torch.jit.save(torch.jit.script(embedder), f"{base_output_dir}/embedder.pt")

UnsupportedNodeError: function definitions aren't supported:
  File "/private/home/pfz/09-videoseal/videoseal-dev/videoseal/modules/unet.py", line 206
    
        # Upward path
        def concat_skip_connect(x): return torch.cat(
        ~~~ <--- HERE
            (x, hiddens.pop() * self.connect_scale), dim=1)
        for ublock in self.ups:
